# Information Retrieval - Pandemic Investigation  
This notebook retrieves abstracts relevant to pandemics and then uses topic modeling to analyze the chosen abstracts.  Three info retrieval techniques are used: Literal Term Matching, TF-IDF, and Latent Semantic Indexing.  These are linear algebra techniques.  
We use the Scikit-Learn library.

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances


#from gensim.models.coherencemodel import CoherenceModel

import TextCleaning

In [2]:
# pull in clean abstracts

with open('../../data/final/processed_dataset_stanford_lemma.pkl', 'rb') as f:
    clean_df = pickle.load(f)

# remove null abstracts and duplicates

df = TextCleaning.remove_nulls(clean_df, "ABSTRACT")
df = TextCleaning.remove_duplicates(df)

df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

0 nulls in  ABSTRACT . These rows removed.
0 duplicate abstracts removed
0 project ID duplicates - not removed


In [3]:
df.head()

,original index,original index,PROJECT_ID,ABSTRACT,FY,FIRST_CHAR,LAST_CHAR,DEPARTMENT,AGENCY,IC_CENTER,...,CFDA_CODE,FY_TOTAL_COST,working_abstract,nchar,Start Char,Field Count,lemma_docs_with_stop,tokened_docs_nostop,tns_bi_tri_docs,final_tokens
0,0,0,89996,"This is a project to explore Game-based, Metap...",2008,"This is a project to explore Game-based, Metap...",.,NSF,NSF,NaN,...,47.076,1999467.0,"This is a project to explore Game-based, Metap...",2057,T,0,"[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan..."
1,1,1,89997,Institution: Franklin Institute Science Museum...,2008,Institution: Franklin Institute Science Museum...,.,NSF,NSF,NaN,...,47.076,1799699.0,"Institution: Science Museum PI: Snyder, Steve...",1913,I,1,"[institution, science, Museum, pi, Snyder, Ste...","[institution, science, museum, pi, steve, proj...","[institution, science, museum, pi, steve, proj...","[institution, science, museum, pi, steve, proj..."
2,2,2,89998,Through programs (including small group conver...,2008,Through programs (including small group conver...,.,NSF,NSF,NaN,...,47.076,1505858.0,Through programs (including small group conver...,1154,T,0,"[program, include, small, group, conversation,...","[program, small, group, conversation, citizen,...","[program, small, group, conversation, citizen,...","[program, small, group, conversation, citizen,..."
3,3,3,89999,In partnership with the American Chemical Soci...,2008,In partnership with the American Chemical Soci...,.,NSF,NSF,NaN,...,47.049,51000.0,In partnership with the American Chemical Soci...,826,I,0,"[partnership, American, Chemical, Society, ACS...","[partnership, american, chemical, society, acs...","[partnership, american, chemical, society, acs...","[partnership, american, chemical, society, acs..."
4,4,4,90000,Amphibian populations around the world are exp...,2008,Amphibian populations around the world are exp...,.,NSF,NSF,NaN,...,47.074,370996.0,Amphibian populations around the world are exp...,1322,A,1,"[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec..."


In [4]:
# input needed for doc-term matrix creation in Scikit-Learn is one string per document (not a list of strings).  
# Original data is already in this form!

docs = df["ABSTRACT"] 

## Functions needed for all info retrieval approaches

In [5]:
# Create query vector 

def create_query(words, terms):
    
    # words: search query words
    # terms: terms in corpus
    
    q = np.zeros(len(terms))  # number of terms

    idx = []
    for word in query_words:
        idx.append(terms.index(word))

    q[idx] = 1
    
    return q

In [6]:
def return_top_abstracts(docs, scores, top_n):
    
    '''
    docs: Series that contains abstract
    scores: scores of abstracts
    top_n: return the top_n abstracts given by idx
    '''
    # sort scores in descending order
    scores_sorted_idx = np.argsort(scores)[::-1]
    
    ix = scores_sorted_idx[:top_n]
    print(ix[0:10])
    
    return ix, docs[ix]
    

In [7]:
def create_result_df(abstracts, scores):
    
    df = pd.DataFrame()
    df["abstracts"] = abstracts
    df["scores"] = scores
    
    return df

## Literal Term Matching - Frequency Count Document-Term Matrix

This will return all abstracts in the corpus with exact word matches to the query.  

Results will be return in sorted order of how high the query scores with each abstract. A high score means more occurences of the query words in the abstract.

In [8]:
# Create document-term matrix based on count frequencies

#Countvectroizer -> document term matrix, all terms from corpus, add ngrams in this function, contact space tracing?
vectorizer = CountVectorizer(ngram_range=(1,2)) #  added parameter to search for bi-grams in addition to single words
doc_term_matrix = vectorizer.fit_transform(docs)

In [9]:
terms = vectorizer.get_feature_names()

### Create Query Words - list the search terms

A query is just a list of words to search for in the corpus.  We will use the same query for all three info retrieval techniques.

In [10]:
# CHANGE QUERY WORDS HERE

query_words = ["pandemic", "contagion", "infection", "quarantine", "test", "virus", "death", "vaccine", "treatment", "public health", "food supply"]

#query_words = ["asymptomatic"] # other example: ['pandemic', influenza', 'mers', 'sars', 'zikv', 'denv', 'hiv', 'aids']

q = create_query(query_words, terms)

In [11]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

f_scores = doc_term_matrix.dot(q)

In [12]:
sum(f_scores >0)  # how many abstracts include at least one of the query words

274230

In [16]:
# sort scores in descending order

f_scores_sorted = np.sort(f_scores)[::-1]
f_scores_sorted[:11]

array([68., 62., 59., 56., 56., 55., 53., 52., 51., 51., 51.])

In [20]:
f_idx, f_top_abstracts = return_top_abstracts(docs, f_scores, 500)  # CHANGE NUMBER OF TOP DOCS RETURNED

[232171  92068 431005 294218 191616 146256 232588 296774 389813 190355]


In [21]:
f_top_abstracts

232171    The hemagglutinin of the 2009 pandemic H1N1 in...
92068     Dengue serotype 1 vaccine development: The liv...
431005    (A) Study pathogenesis and pathophysiology of ...
294218    The epidemiological success of pandemic and ep...
191616    (I) Disease models and pathogenesis Syrian gol...
                                ...                        
206843    DESCRIPTION (provided by applicant): CD8+ T-ce...
125697    DESCRIPTION (provided by applicant): Dengue (D...
207570    DESCRIPTION (provided by applicant): Anorexia ...
90704     Summary: This clinical research project is for...
168056    DESCRIPTION (provided by applicant): Hepatitis...
Name: ABSTRACT, Length: 500, dtype: object

In [22]:
f_top_abstracts.iloc[0]

'The hemagglutinin of the 2009 pandemic H1N1 influenza virus is a derivative of and is antigenically related to classical swine but not to seasonal human H1N1 viruses. We compared the A/California/7/2009 (CA/7/09) virus recommended by the WHO as the reference virus for vaccine development, with two classical swine influenza viruses A/swine/Iowa/31 (sw/IA/31) and A/New Jersey/8/1976 (NJ/76) to establish the extent of immunologic cross-reactivity and cross-protection in animal models. Primary infection with 2009 pandemic or NJ/76 viruses elicited antibodies against the CA/7/09 virus and provided complete protection from challenge with this virus in ferrets; the response in mice was variable and conferred partial protection. Although ferrets infected with sw/IA/31 virus developed low titers of cross-neutralizing antibody, they were protected from pulmonary replication of the CA/7/09 virus. The data suggest that prior exposure to antigenically related H1N1 viruses of swine-origin, by prior

In [23]:
f_df = create_result_df(docs, f_scores)

In [24]:
f_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",2.0
1,Institution: Franklin Institute Science Museum...,1.0
2,Through programs (including small group conver...,0.0
3,In partnership with the American Chemical Soci...,0.0
4,Amphibian populations around the world are exp...,2.0
...,...,...
543428,The Title IV-E Prevention Services Clearinghou...,1.0
543429,This mixed-methods study seeks to deepen our u...,0.0
543430,The purpose of this project is to examine the ...,0.0
543431,The 2014 Child Care and Development Block Gran...,0.0


## TF-IDF Document-Term Matrix

This approach is similar to Literal Term Matching using frequency counts in the document-term matrix.  However, instead of using frequency counts, the entries of the document-term matrix are weighted using TF-IDF.

In [25]:
# Find doc-term matrix using TF-IDF weighting
#Added this line
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer(ngram_range=(1,2)) #  added parameter to search for bi-grams in addition to single words)
tf_idf = tf_idf_vectorizer.fit_transform(docs)

In [26]:
tf_idf_terms = tf_idf_vectorizer.get_feature_names()  # these terms are the same as the terms created from the 
                                                      # frequency count document-term matrix, so we do not need to
                                                      # recreate the query vector.

In [27]:
# calculate the score for each document against the query. Docs with more occurences of the query words 
# will score higher

tf_idf_scores = tf_idf.dot(q)

In [28]:
sum(tf_idf_scores >0)   # how many abstracts include at least one of the query words

274230

In [29]:
# sort scores in descending order

tf_idf_scores_sorted = np.sort(tf_idf_scores)[::-1]
tf_idf_scores_sorted[:11]

array([0.63783573, 0.58029174, 0.54346076, 0.54263703, 0.53494587,
       0.53361225, 0.52932968, 0.51473581, 0.5019565 , 0.50131183,
       0.49978857])

In [62]:
tfidf_idx, tfidf_top_abstracts = return_top_abstracts(docs, tf_idf_scores, 500)  # CHANGE NUMBER OF TOP DOCS RETURNED

[232171  61937 191946 431005 296774 144405 127383 110836 233437 523831]


In [63]:
tfidf_top_abstracts

232171    The hemagglutinin of the 2009 pandemic H1N1 in...
61937     DESCRIPTION (provided by applicant): Influenza...
191946    Influenza A viruses (IAV) are significant huma...
431005    (A) Study pathogenesis and pathophysiology of ...
296774    Influenza A viruses (IAV) are significant huma...
                                ...                        
18043     DESCRIPTION (provided by applicant): We have i...
150961    Simple contagion processes underlie various ph...
59238     DESCRIPTION (provided by applicant): The overa...
442851    While the outbreak of ZIKV in the Americas is ...
39482     DESCRIPTION (provided by applicant): Sexually ...
Name: ABSTRACT, Length: 500, dtype: object

In [64]:
tf_idf_df = create_result_df(docs, tf_idf_scores)

In [65]:
tf_idf_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.021442
1,Institution: Franklin Institute Science Museum...,0.012610
2,Through programs (including small group conver...,0.000000
3,In partnership with the American Chemical Soci...,0.000000
4,Amphibian populations around the world are exp...,0.047313
...,...,...
543428,The Title IV-E Prevention Services Clearinghou...,0.011972
543429,This mixed-methods study seeks to deepen our u...,0.000000
543430,The purpose of this project is to examine the ...,0.000000
543431,The 2014 Child Care and Development Block Gran...,0.000000


## Latent Semantic Indexing (LSI) Approach

LSI Uses the TF-IDF matrix.  LSI is a tecnique that utilizes a truncated Singular Value Decomposition of the document-term matrix.  Basically, LSI still returns relevant documents to the query; however some of the documents returned may not include the exact search terms!  LSI is finding the latent or hidden relationships in the terms.  

In [34]:
# Find the Truncated SVD of the TF-IDF matrix

lsa = TruncatedSVD(n_components=10 , random_state=1)  # CHANGE THE NUMBER OF COMPONENTS - NOTE: MORE COMPONENTS 
                                                      # GIVES YOU A MORE ACCURATE APPROXIMATION OF THE DOC-TERM 
                                                      # MATRIX, BUT IS ALSO MORE EXPENSIVE AND MAY NOT LEAD TO THE 
                                                      # BEST INFO RETRIEVAL RESULTS.
USigma = lsa.fit_transform(tf_idf)
Vtrans = lsa.components_

In [35]:
# transform query to be in same space as documents

q = q.reshape(1,-1)
qhat = lsa.transform(q)

In [36]:
print(qhat.shape)
print(USigma.shape)
print(Vtrans.shape)

(1, 10)
(543433, 10)
(10, 17871980)


In [38]:
lsa_scores = pairwise_distances(qhat, USigma, metric='cosine', n_jobs=39)  # CHANGE N_JOBS TO BE NUMBER OF CORES - 1

In [39]:
lsa_scores.shape

(1, 543433)

In [40]:
lsa_scores

array([[0.99798616, 1.02524077, 0.94087696, ..., 0.87714663, 0.80300928,
        0.83415476]])

In [41]:
type(lsa_scores)

numpy.ndarray

In [42]:
sum(lsa_scores[0] > 0)  # how many abstracts scored above 0

543433

In [43]:
lsa_scores[0]

array([0.99798616, 1.02524077, 0.94087696, ..., 0.87714663, 0.80300928,
       0.83415476])

In [46]:
# sort scores in descending order

lsa_scores_sorted = np.sort(lsa_scores[0])[::-1]
lsa_scores_sorted[:11]

array([1.22881454, 1.22216184, 1.19503948, 1.19458075, 1.19100107,
       1.18562919, 1.18022599, 1.17727443, 1.16565007, 1.1588575 ,
       1.15776969])

In [48]:
lsa_idx, lsa_top_abstracts = return_top_abstracts(docs, lsa_scores[0], 500)  # CHANGE NUMBER OF TOP DOCS RETURNED

[108250 102828 105654 157495 103750 159206 157069   3658  95998  95693]


In [49]:
lsa_top_abstracts

108250    Wallace Fowler/National Space Grant Foundation...
102828    ROBERTA JOHNSON/UNIVERSITY CORPORATION FOR ATM...
105654    THE MULTI-MISSION ARCHIVE AT SPACE SCIENCE INS...
157495    TAS::80 0119::TAS UNIVERSITY CORPORATION AT MO...
103750    GLOBAL CLIMATE CHANGE EDUCATION EXCHANGE (GCCE...
                                ...                        
298348    This CC-NIE Integration project performed at t...
153863    This award supports travel for junior research...
535200    The Center for Discrete Mathematics and Theore...
5998      The Proteomics and Functional Genomics (PFG) S...
472218    Scientific conference on Quantitative Genetics...
Name: ABSTRACT, Length: 500, dtype: object

In [50]:
lsa_df = create_result_df(docs, lsa_scores[0])

In [51]:
lsa_df

,abstracts,scores
0,"This is a project to explore Game-based, Metap...",0.997986
1,Institution: Franklin Institute Science Museum...,1.025241
2,Through programs (including small group conver...,0.940877
3,In partnership with the American Chemical Soci...,0.982410
4,Amphibian populations around the world are exp...,0.753694
...,...,...
543428,The Title IV-E Prevention Services Clearinghou...,0.892304
543429,This mixed-methods study seeks to deepen our u...,0.802961
543430,The purpose of this project is to examine the ...,0.877147
543431,The 2014 Child Care and Development Block Gran...,0.803009


## Create pandemics corpus

We use the results of our three information retrieval techniques to create a new, smaller corpus that only contains abstracts relevant to the query. 

In [52]:
docs_ix = np.concatenate([f_idx, tfidf_idx, lsa_idx])

In [53]:
docs_idx = np.unique(docs_ix)

In [54]:
docs_idx.shape

(1079,)

In [55]:
lim_docs = [docs[i] for i in docs_idx]

In [56]:
# print out the abstracts of the pandemics corpus -- long text output!

lim_docs

['The University of Colorado is establishing a Distributed Center for STEM Education Research and Transformation that integrates education projects across the campus.  The Center addresses the three themes outlined in the National Academy of Science report  Rising Above the Gathering Storm . These are: (1) Teachers in K-12 education (10,000 Teachers, 10 Million Minds), (2) Research (Sowing Seeds), and (3) Higher Education (Best and Brightest).  This Distributed Center involves eight traditional departments in three colleges and schools, including: Education, Life Sciences, Mathematics, Physical Sciences, and Engineering.  Existing education projects being integrated into the center include: ADVANCE (Increasing the Participation and Advancement of Women in Academic Science and Engineering Careers); Course Curriculum and Laboratory Improvement projects; Robert Noyce Teaching Scholarship project; Integrative Graduate Education and Research Traineeship projects; and Research and Evaluation

## Topic Modeling with relevant pandemic abstracts

In [57]:
# Create a TF-IDF document-term matrix for the pandemics corpus 

# TRY DIFFERENT PARAMETERS IN THE TF-IDF DOC-TERM MATRIX SET-UP
nmf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2) , #tfdi vector see for contact, run 1-2, bigrams - 2 word phrases occured right after the other as one token, if we had n_gram range from1-3, unigrams - 1 word token, bigrams and trigrams
                                 max_df=0.4, min_df=3, lowercase=True) #, max_features=int(len(lim_docs)/2))

nmf_tf_idf = nmf_vectorizer.fit_transform(lim_docs)

In [58]:
# topic modeling with NMF

nmf_model = NMF(n_components=10, random_state=1)  # TRY DIFFERENT NUMBERS OF TOPICS
W = nmf_model.fit_transform(nmf_tf_idf)
H = nmf_model.components_

In [59]:
# TRY TOPIC MODELING WITH LDA



In [60]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
        

In [61]:
print_topics(nmf_model, nmf_vectorizer, 10)


Topic 0:
('hiv', 1.0932107519090526)
('cells', 0.6583327101234124)
('siv', 0.6439856845181973)
('cell', 0.6356042871525992)
('responses', 0.4104283692405834)
('cd8', 0.4022573787312421)
('cd4', 0.3826426189152297)
('immune', 0.33557426025786075)
('macaques', 0.32612105111168443)
('specific', 0.319934166837354)

Topic 1:
('students', 0.9279905493611318)
('engineering', 0.3780319671334387)
('undergraduate', 0.3698124557323009)
('graduate', 0.34701487450381535)
('science', 0.3339522534706875)
('materials', 0.3174419723560004)
('program', 0.3069895538966148)
('faculty', 0.27102139923410823)
('conference', 0.26692012450028013)
('reu', 0.25781446625914983)

Topic 2:
('influenza', 1.1460416877850454)
('pandemic', 0.5841800538190307)
('influenza virus', 0.40094340370168446)
('1918', 0.38422269506046636)
('viruses', 0.327922739070049)
('h1n1', 0.30439384706939665)
('influenza viruses', 0.2509980892783758)
('pandemics', 0.20588240507432093)
('2009', 0.2022179287358947)
('h5n1', 0.18447070979654